In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [ ]:
losses = []
torch.manual_seed(42)

In [ ]:
torch.cuda.is_available()

In [ ]:
x = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float32)
y = torch.tensor([0,1,1,0], dtype = torch.float32)

In [ ]:
class xor(nn.Module):
    def __init__(self):
        super(xor,self).__init__()
        self.l1 = nn.Linear(2,2,bias=True)
        self.sig1 = nn.Sigmoid()
        self.l2 = nn.Linear(2,1,bias = True)
        self.sig2 = nn.Sigmoid()
    def forward(self, x):
        x = self.l1(x)
        x = self.sig1(x)
        x = self.l2(x)
        #x = self.sig2(x)
        return x

In [ ]:
class mydata(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, id):
        return self.x[id].to(torch.device("cuda")), self.y[id].to(torch.device("cuda"))

In [ ]:
data = mydata(x,y)
batch = 1
loader = DataLoader(data, batch_size = batch, shuffle = True)
device = torch.device("cuda")
model = xor().to(device)
print(model)

In [ ]:
loss = torch.nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr = 0.03)


In [ ]:
def train_one_epoch(index):
    total = 0
    for i, data in enumerate(loader):
        inputs, labels = data
        optim.zero_grad()
        outputs = model(inputs)
        l = loss(outputs.flatten(), labels)
        l.backward()
        optim.step()
        total += l.item()
    return total/len(loader)/batch

In [ ]:
epochs = 10000
for epoch in range(epochs):
    model.train(True)
    l = train_one_epoch(model)
    losses.append(l)
    if epoch%1000==0:
        print(f"epoch: {epoch}\nLoss:{l}\n")

In [ ]:
plt.plot(losses)

In [ ]:
def sigmoid(x):
    return 1.0/(1+torch.exp(-x))


In [ ]:
test = torch.tensor([0,0], dtype = torch.float32).to(device)
answer = 1
layer1 = torch.matmul(model.l1.weight, test) + model.l1.bias
layer1 = sigmoid(layer1)
layer2 = torch.matmul(model.l2.weight, layer1) + model.l2.bias
layer2 = sigmoid(layer2)
print(layer2)